In [1]:
import torch

import torch


class GoGame:
    """
    Go game class.
    This class implements the Go game logic to be used for training the neural network.
    """

    def __init__(self, board_size=19) -> None:
        """
        Initializes the Go game with the given board size.
        Args:
            board_size (int): Size of the Go board (default is 19).
        """

        self.board_size = board_size
        self.board = torch.zeros((board_size, board_size, 2), dtype=torch.bool)

    def place_stone(self, x, y, dim) -> None:
        """
        Places a stone of the specified color at the given position (x, y) on the board.
        Args:
            x (float): X-coordinate of the position.
            y (float): Y-coordinate of the position.
            dim (float): Color of the stone (0 for black, 1 for white).
        """
        self.board[x][y][dim] = True
        self.__remove_dead_stones(x, y, dim)

    def __find_group_to_be_remove(self, x, y, dim) -> set:
        """
        Finds a group of stones of the specified color to be removed from the board.
        Args:
            x (float): X-coordinate of the position.
            y (float): Y-coordinate of the position.
            dim (float): Color of the stone (0 for black, 1 for white).
        Returns:
            set: Set of positions of stones in the group.
        """
        # Create a set to store positions of stones in the group
        group = set()

        # Create a set to store positions of stones to be checked
        to_be_check = set([(x, y)])

        # if the position has no stone, return an empty group
        if self.board[x][y][dim] == False:
            return group

        # Iterate through the positions to be checked
        while len(to_be_check) > 0:
            # Pop the position to be checked
            x, y = to_be_check.pop()

            # Check if the position is already in the group
            if (x, y) in group:
                continue

            # if the position is opponent's stone, continue
            if self.board[x][y][-dim + 1] == True:
                continue

            # if the position is empty, it has liberty, so this group is not dead
            if self.board[x][y][-dim + 1] == False and self.board[x][y][dim] == False:
                return set()

            # Add the position to the group
            group.add((x, y))

            # Add the neighbors to the positions to be checked
            for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                nx, ny = x + dx, y + dy
                if nx < 0 or nx >= self.board_size or ny < 0 or ny >= self.board_size:
                    continue
                to_be_check.add((nx, ny))

        return group

    def __remove_dead_stones(self, x, y, dim) -> None:
        """
        Removes dead stones of the specified color from the board.

        Args:
            x (float): X-coordinate of the position.
            y (float): Y-coordinate of the position.
            dim (float): Color of the stone (0 for black, 1 for white).
        """

        # Determine the opponent's dimension
        opponent_dim = -dim + 1

        # Find groups of opponent stones to be removed
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            # find neighbor stones of the same color
            nx, ny = x + dx, y + dy
            if nx < 0 or nx >= self.board_size or ny < 0 or ny >= self.board_size:
                continue
            if self.board[nx][ny][opponent_dim] == False:
                continue
            # Remove the group if it has no liberty
            group = self.__find_group_to_be_remove(nx, ny, opponent_dim)
            for gx, gy in group:
                self.board[gx][gy][opponent_dim] = False

    def get_board(self) -> torch.Tensor:
        """
        Returns the current game board.
        Returns:
            torch.Tensor: Current game board.
        """
        return self.board

    def reset(self) -> None:
        """
        Resets the game board to the initial state.
        """
        self.board = torch.zeros(
            (self.board_size, self.board_size, 2), dtype=torch.bool
        )

/home/alanchou/micromamba/envs/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import csv
import os
import tqdm
import math


class Converter:
    def __init__(self, path) -> None:
        self.path = path
        self.goGame = GoGame()
        self.char2idx = {c: i for i, c in enumerate("abcdefghijklmnopqrs")}

        # Load data from CSV file
        with open(self.path, newline="") as csvfile:
            reader = csv.reader(csvfile, delimiter=",")
            # Read row by row
            self.data = list(reader)  # dtype: list[str]
            self.length = len(self.data)

    def __step(self, step):
        """
        Perform a step in the game based on the given input step.
        Args:
            step (str): A str containing player, x-coordinate, and y-coordinate information.
        """
        dim = 0 if step[0] == "B" else 1
        x = self.char2idx[step[2]]
        y = self.char2idx[step[3]]
        self.goGame.place_stone(x, y, dim)

    def __transform(self, data):
        """
        Transform data from CSV data to boards and add other features.
        Args:
            data (list): List of steps in the game.
        Returns:
            torch.Tensor: Transformed data.
        """
        transformed_data = []

        filename = data[0]

        for i in range(2, len(data)):
            self.__step(data[i])
            transformed_data.append(self.goGame.get_board().clone())

        return filename, torch.stack(transformed_data)
    
    def convert(self):
        """
        Convert data from CSV data and store in disk.
        """

        num_dirs = int(math.sqrt(self.length))

        for i in range(num_dirs):
            sub_dir_path = os.path.join("data/preprocessed data/", f'subdir_{i}')
            os.makedirs(sub_dir_path, exist_ok=True)

        for data in tqdm.tqdm(self.data):
            filename, transformed_data = self.__transform(data)
            subdir = f'subdir_{int(filename[2:])%num_dirs}'
            torch.save(transformed_data, os.path.join("data/preprocessed data/", subdir, filename + ".pt"))

if os.path.exists("data/preprocessed data") == False:
    os.mkdir("data/preprocessed data")
converter = Converter("data/train/dan_train.csv")
converter.convert()


100%|██████████| 100160/100160 [36:07<00:00, 46.20it/s]
